## Get Data .zip from Kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/EuroSAT_224.zip /content/

In [ ]:
!unzip -q EuroSAT_224.zip

In [ ]:
from google.colab import files
files.upload() #upload kaggle.json for the API key

In [ ]:
#!cp /content/EuroSAT_224.zip /content/drive/MyDrive/

In [ ]:
!pip install -q kaggle # install kaggle api client

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d apollo2506/eurosat-dataset/ # download dataset take url after dataset/


Dataset URL: https://www.kaggle.com/datasets/apollo2506/eurosat-dataset/versions/
License(s): CC0-1.0
 99% 2.03G/2.04G [00:32<00:00, 122MB/s]
100% 2.04G/2.04G [00:32<00:00, 68.4MB/s]


In [ ]:
!unzip -q eurosat-dataset.zip # unzip in /content/

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.models import vit_b_16, ViT_B_16_Weights
from tqdm.notebook import tqdm
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import os

## Data Preprocessing & Loaders

In [ ]:
train_df = pd.read_csv('/content/content/EuroSAT_224/train.csv')
val_df = pd.read_csv('/content/content/EuroSAT_224/validation.csv')
test_df = pd.read_csv('/content/content/EuroSAT_224/test.csv')

In [ ]:
test_df.iloc[0]['Filename'], test_df.iloc[0]['Label']

('AnnualCrop/AnnualCrop_1275.jpg', 0)

In [ ]:
def preprocess_and_save_images(df, source_dir, target_dir, target_size=(224, 224)):
    # Create target directory if it doesn't exist
    os.makedirs(target_dir, exist_ok=True)

    for _, row in tqdm(df.iterrows(), total=len(df)):
        img_path = os.path.join(source_dir, row['Filename'])
        img = Image.open(img_path).convert("RGB")
        img_resized = img.resize(target_size)

        # Save the resized image to the target directory, maintaining the directory structure
        target_path = os.path.join(target_dir, row['Filename'])
        os.makedirs(os.path.dirname(target_path), exist_ok=True)
        img_resized.save(target_path)

path = '/content/EuroSAT/'
new_path = '/content/EuroSAT_224/'
# Preprocess and save images for each split
#preprocess_and_save_images(train_df, path, new_path)
#preprocess_and_save_images(val_df, path, new_path)
#preprocess_and_save_images(test_df, path, new_path)

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, num_classes, transform=None):
        self.img_labels = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.num_classes = num_classes

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image_info = self.img_labels.iloc[idx]
        img_path = os.path.join(self.img_dir, image_info['Filename'])
        image = Image.open(img_path).convert("RGB")
        label = image_info['Label']
        label_onehot = torch.zeros(self.num_classes)
        label_onehot[label] = 1.0

        if self.transform:
            image = self.transform(image)

        return image, label_onehot

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
path = '/content/content/EuroSAT_224/'
train_dataset = CustomImageDataset(dataframe=train_df, img_dir=path, num_classes=10, transform=transform)
val_dataset = CustomImageDataset(dataframe=val_df, img_dir=path, num_classes=10, transform=transform)
test_dataset = CustomImageDataset(dataframe=test_df, img_dir=path, num_classes=10, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
for input, label in train_loader:
    print(input.shape, label.shape)
    break

torch.Size([32, 3, 224, 224]) torch.Size([32, 10])


In [ ]:
from torchvision.models import vit_b_16, swin_b, ViT_B_16_Weights, Swin_B_Weights

#vit_weights = ViT_B_16_Weights.DEFAULT
#model = vit_b_16(pretrained=True) # get pre-trained model
vit_model = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
vit_model.cpu()
swin_model = swin_b(weights=Swin_B_Weights.DEFAULT)

# replace classification head
num_classes = 4
vit_model.heads.head = nn.Linear(in_features=vit_model.heads.head.in_features, out_features=num_classes)
swin_model.head = nn.Linear(in_features=swin_model.head.in_features, out_features=num_classes)

# Step 3: Freeze all layers except the last one
for param in vit_model.parameters():
    param.requires_grad = False
for param in vit_model.heads.head.parameters():
    param.requires_grad = True

# Step 3: Freeze all layers except the last one
for param in swin_model.parameters():
    param.requires_grad = False
for param in swin_model.head.parameters():
    param.requires_grad = True

model = vit_model

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:07<00:00, 44.5MB/s]
Downloading: "https://download.pytorch.org/models/swin_b-68c6b09e.pth" to /root/.cache/torch/hub/checkpoints/swin_b-68c6b09e.pth
100%|██████████| 335M/335M [00:02<00:00, 157MB/s]


In [ ]:
model(input).shape

torch.Size([32, 10])

In [ ]:
from torch.optim.lr_scheduler import StepLR
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)  # Move the model to the device

# Define the loss function
criterion = nn.CrossEntropyLoss()

#optimizer = optim.Adam(net.parameters(), lr=0.0001)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
#scheduler = StepLR(optimizer, step_size=3, gamma=0.1)
epochs=10
for epoch in range(epochs):  # loop over the dataset multiple times
    model.train()
    prog_bar = tqdm(train_loader, desc=f"Epoch: {epoch+1}/{epochs}")
    for inputs, labels in prog_bar: # desc =str(epoch)+"/"+str(epochs), miniters=int(50),unit='img'
        # get the inputs; data is a list of [inputs, labels]
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Update the progress bar with the current loss
        prog_bar.set_postfix(**{'loss': np.round(loss.data.cpu().detach().numpy(),5)})

    # put on eval and run validation dataset every 5 epochs
    if epoch%3 == 0:
        all_preds = []
        all_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Get predictions
                _, preds = torch.max(outputs, dim=1) # get column of max value -> 0-15
                _, lab = torch.max(labels, dim=1)

                # Append predictions and labels
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(lab.cpu().numpy())

        # calc accuracy
        i = 0
        for y, y_hat in zip(all_labels, all_preds):
            if y == y_hat:
                i += 1
        print(f"Accuracy: {i/len(all_labels)}")



Epoch: 1/10:   0%|          | 0/591 [00:00<?, ?it/s]

Accuracy: 0.9366666666666666


Epoch: 2/10:   0%|          | 0/591 [00:00<?, ?it/s]

Epoch: 3/10:   0%|          | 0/591 [00:00<?, ?it/s]

Epoch: 4/10:   0%|          | 0/591 [00:00<?, ?it/s]

...

Accuracy: 0.9585185185185185


Epoch: 8/10:   0%|          | 0/591 [00:00<?, ?it/s]

Epoch: 9/10:   0%|          | 0/591 [00:00<?, ?it/s]

Epoch: 10/10:   0%|          | 0/591 [00:00<?, ?it/s]

Accuracy: 0.960925925925926


In [ ]:
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)  # Move the model to the device

all_preds = []
all_labels = []

for inputs, labels in tqdm(test_loader, desc=f"Epoch: {epoch}/{epochs}"):
    inputs = inputs.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(inputs)

    # Get predictions
    _, preds = torch.max(outputs, dim=1) # get column of max value -> 0-15
    _, lab = torch.max(labels, dim=1)

    # Append predictions and labels
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(lab.cpu().numpy())

# calc accuracy
i = 0
for y, y_hat in zip(all_labels, all_preds):
    if y == y_hat:
        i += 1
print(f"Accuracy: {i/len(all_labels)}")


Epoch: 9/10:   0%|          | 0/85 [00:00<?, ?it/s]

Accuracy: 0.9666666666666667
